In [1]:
import pandas as pd
import numpy as np
from kiteconnect import KiteConnect
import pandas as pd
from time import sleep
from kiteconnect import KiteConnect, KiteTicker
import login
from datetime import datetime, timedelta
import time
import pytz
import pymongo
import threading

In [2]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['algostrat']
collection = db['trade_logs']

# connect to kite api
user_name = "ZT8131"
password = "GoalFi@123"
totp = "EEJXULM7H43NHJ2KNYJJQ7ZK73INQG35"
api_key = "50ck6xue25zuf01z"
api_secret = "oprzecf4hej9hbs6m621viawto8erfyw"

kite = KiteConnect(api_key=api_key)
request_token = login.kiteLogin(user_name, password, totp, api_key)
data = kite.generate_session(request_token, api_secret)
kite.set_access_token(data["access_token"])

In [3]:
instrument_dump = kite.instruments()   # get instruments dump from NSE
instrument_df = pd.DataFrame(instrument_dump)  # dump it to a dataframe
instrument_df['expiry']=pd.to_datetime(instrument_df['expiry'])
print(instrument_df.columns)

Index(['instrument_token', 'exchange_token', 'tradingsymbol', 'name',
       'last_price', 'expiry', 'strike', 'tick_size', 'lot_size',
       'instrument_type', 'segment', 'exchange'],
      dtype='object')


In [4]:
def get_instrument_token(instrument_df, symbol):
    return instrument_df[(instrument_df['tradingsymbol'] == symbol)&(instrument_df['exchange'] == 'NSE')]['instrument_token'].values[0]

In [5]:
constitutions = pd.read_csv('2024-08-14.csv')
#sort data based on NSE Ticker
constitutions = constitutions.sort_values('NSE Ticker')
#remove rows with same NSE Ticker
constitutions = constitutions.drop_duplicates(subset='NSE Ticker', keep='first')
print(len(constitutions))
# print(data)

#get last one month data for each stock from column NSE Ticker
portfolio=pd.DataFrame()
for index, row in constitutions.iterrows():
    print(row['NSE Ticker'])
    instrument_token = get_instrument_token(instrument_df, row['NSE Ticker'])
    print(instrument_token)

    #get last one month data for each stock
    to_date = datetime.now().date()
    from_date = to_date - timedelta(days=30)
    from_date = from_date.strftime('%Y-%m-%d')
    to_date = to_date.strftime('%Y-%m-%d')
    print(from_date, to_date)
    data = kite.historical_data(instrument_token, from_date, to_date, 'day')
    #change the data to dataframe
    data = pd.DataFrame(data)
    data['pct_change'] = data['close'].pct_change()
    data = data[['date', 'pct_change']]
    data['date'] = pd.to_datetime(data['date'])
    data = data.set_index('date')
    print(data)

    #push the data to portfolio
    portfolio[row['NSE Ticker']] = data['pct_change']*100
    print(portfolio)
    
        

6
CDSL
5420545
2024-08-27 2024-09-26
                           pct_change
date                                 
2024-08-27 00:00:00+05:30         NaN
2024-08-28 00:00:00+05:30   -0.027009
2024-08-29 00:00:00+05:30   -0.041420
2024-08-30 00:00:00+05:30    0.002445
2024-09-02 00:00:00+05:30   -0.012893
2024-09-03 00:00:00+05:30    0.003565
2024-09-04 00:00:00+05:30   -0.000985
2024-09-05 00:00:00+05:30   -0.002113
2024-09-06 00:00:00+05:30   -0.010974
2024-09-09 00:00:00+05:30   -0.020621
2024-09-10 00:00:00+05:30   -0.013223
2024-09-11 00:00:00+05:30    0.003581
2024-09-12 00:00:00+05:30    0.013941
2024-09-13 00:00:00+05:30    0.002322
2024-09-16 00:00:00+05:30    0.045785
2024-09-17 00:00:00+05:30    0.007995
2024-09-18 00:00:00+05:30    0.032893
2024-09-19 00:00:00+05:30    0.004621
2024-09-20 00:00:00+05:30    0.018232
2024-09-23 00:00:00+05:30   -0.011114
2024-09-24 00:00:00+05:30   -0.015642
2024-09-25 00:00:00+05:30   -0.001002
2024-09-26 00:00:00+05:30   -0.008187
             

In [6]:
#remove the first row of the portfolio
portfolio = portfolio.iloc[1:]
display(portfolio)

,CDSL,INFY,JUBLPHARMA,JYOTHYLAB,PCBL,SHYAMMETL
date,,,,,,
2024-08-28 00:00:00+05:30,-2.700854,2.052524,-1.374021,-2.493923,2.044648,-2.677804
2024-08-29 00:00:00+05:30,-4.141972,-0.296529,-0.539986,-0.369344,-1.564097,-1.501956
2024-08-30 00:00:00+05:30,0.244516,0.535340,-0.157446,0.055607,1.547409,2.171963
2024-09-02 00:00:00+05:30,-1.289288,1.070124,-1.141925,1.583920,-0.204541,0.589453
2024-09-03 00:00:00+05:30,0.356538,-1.183507,0.874587,0.692988,3.258865,-0.012468
2024-09-04 00:00:00+05:30,-0.098491,-0.968448,1.183271,0.316943,1.270345,3.148575
2024-09-05 00:00:00+05:30,-0.211260,0.556581,1.067040,0.027081,-3.724030,0.362669
2024-09-06 00:00:00+05:30,-1.097350,-1.619119,1.487683,-1.380742,-2.880700,-3.920742
2024-09-09 00:00:00+05:30,-2.062076,-0.378579,8.359166,0.228770,-0.167697,2.068576


In [7]:
def calculate_portfolio_std(portfolio, correlation_matrix):
    n = len(portfolio)
    portfolio_variance = 0
    
    for i in range(n):
        for j in range(n):
            asset_i = portfolio.index[i]
            asset_j = portfolio.index[j]
            
            # Ensure the assets exist in the portfolio DataFrame
            if asset_i in portfolio.index and asset_j in portfolio.index:
                weight_i = portfolio.loc[asset_i, 'Weight']/100
                weight_j = portfolio.loc[asset_j, 'Weight']/100
                std_i = portfolio.loc[asset_i, 'std_dev']
                std_j = portfolio.loc[asset_j, 'std_dev']
                correlation_ij = correlation_matrix.loc[asset_i, asset_j]
                
                portfolio_variance += weight_i * weight_j * std_i * std_j * correlation_ij
    
    portfolio_std_dev = np.sqrt(portfolio_variance)
    return portfolio_std_dev

In [8]:
#caluculate the correlation for portfolio
correlation = portfolio.corr()
#create a dataframe for correlation
correlation = pd.DataFrame(correlation)
# print(correlation)
display(correlation)
#make a heatmap for correlation


#calculate standard deviation of portfolio
std_dev = portfolio.std()
# print(std_dev)

#add std_dev to constitution dataframe
constitutions['std_dev'] = std_dev.values
print(constitutions)
#set NSE Ticker as index   
constitutions = constitutions.set_index('NSE Ticker')



,CDSL,INFY,JUBLPHARMA,JYOTHYLAB,PCBL,SHYAMMETL
CDSL,1.000000,-0.201199,0.094926,-0.185248,0.433969,0.422700
INFY,-0.201199,1.000000,-0.090784,-0.125686,-0.209747,0.131631
JUBLPHARMA,0.094926,-0.090784,1.000000,-0.294177,-0.138788,0.476943
JYOTHYLAB,-0.185248,-0.125686,-0.294177,1.000000,-0.038992,-0.008288
PCBL,0.433969,-0.209747,-0.138788,-0.038992,1.000000,0.288076
SHYAMMETL,0.422700,0.131631,0.476943,-0.008288,0.288076,1.000000


  Rebalance Date  NSE Ticker                            Security Name  \
5    14 Aug 2024        CDSL  Central Depository Services (India) Ltd   
4    14 Aug 2024        INFY                              Infosys Ltd   
3    14 Aug 2024  JUBLPHARMA                    Jubilant Pharmova Ltd   
2    14 Aug 2024   JYOTHYLAB                          Jyothy Labs Ltd   
1    14 Aug 2024        PCBL                                 PCBL Ltd   
0    14 Aug 2024   SHYAMMETL            Shyam Metalics and Energy Ltd   

                     Segment  Weight   std_dev  
5  Stock Exchanges & Ratings      20  1.897389  
4   IT Services & Consulting      20  1.133121  
3            Pharmaceuticals      15  3.654427  
2  FMCG - Household Products      15  1.431798  
1        Commodity Chemicals      15  2.459843  
0               Iron & Steel      15  2.313809  


In [9]:
import plotly.express as px
fig = px.imshow(correlation)
# fig.update_layout(title='Correlation Matrix', width=1200, height=1200)
# show correlation value on the boxes inside the heatmap
fig.update_traces(showscale=False)
for i in range(len(correlation)):
    for j in range(len(correlation)):
        fig.add_annotation(x=i, y=j, text=str(round(correlation.iloc[i, j], 2)), showarrow=False)


fig.show()


In [10]:
print(constitutions)
print(correlation)

           Rebalance Date                            Security Name  \
NSE Ticker                                                           
CDSL          14 Aug 2024  Central Depository Services (India) Ltd   
INFY          14 Aug 2024                              Infosys Ltd   
JUBLPHARMA    14 Aug 2024                    Jubilant Pharmova Ltd   
JYOTHYLAB     14 Aug 2024                          Jyothy Labs Ltd   
PCBL          14 Aug 2024                                 PCBL Ltd   
SHYAMMETL     14 Aug 2024            Shyam Metalics and Energy Ltd   

                              Segment  Weight   std_dev  
NSE Ticker                                               
CDSL        Stock Exchanges & Ratings      20  1.897389  
INFY         IT Services & Consulting      20  1.133121  
JUBLPHARMA            Pharmaceuticals      15  3.654427  
JYOTHYLAB   FMCG - Household Products      15  1.431798  
PCBL              Commodity Chemicals      15  2.459843  
SHYAMMETL                Iron & S

In [11]:
#calculate portfolio std deviation
portfolio_std_dev=calculate_portfolio_std(constitutions,correlation)
print(portfolio_std_dev)

1.0389477826170526
